In [1]:
#!pip3 install pandas
#!pip3 install numpy
#!pip3 install sentence_transformers

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

project_root = os.path.abspath("..")  
if project_root not in sys.path:
    sys.path.append(project_root)

from src.retriever.vector_store import VectorStore

DATA_PATH = os.path.join(project_root, "data", "clean_books.csv")
INDEX_PATH = os.path.join(project_root, "models", "faiss_index.bin")
META_PATH  = os.path.join(project_root, "models", "metadata.pkl")

print("Data path:", DATA_PATH)
print("Index path:", INDEX_PATH)
print("Meta path:", META_PATH)

df = pd.read_csv(DATA_PATH)
print("Rows:", len(df))
print(df.columns.tolist())

model = SentenceTransformer("all-MiniLM-L6-v2")

vectors = []
metas = []

for _, row in df.iterrows():
    title = str(row["Title"]).strip()
    author = str(row["Author"]).strip()
    genres = row["genres"]                
    rating = float(row["average_rating"])
    retrieval_text = str(row["retrieval_text"]).strip()

    text = retrieval_text

    emb = model.encode(text)
    vectors.append(emb)

    metas.append(
        {
            "title": title,
            "author": author,
            "genres": genres,
            "rating": rating,
            "retrieval_text": retrieval_text,
        }
    )

vectors = np.array(vectors)
print("Embedding shape:", vectors.shape)

vs = VectorStore(dim=vectors.shape[1])
vs.add(vectors, metas)

vs.save(index_path=INDEX_PATH, meta_path=META_PATH)

print("✅ Rebuild complete!")

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data path: /Users/biancaleoveanu/Desktop/rag-book-recommender/data/clean_books.csv
Index path: /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Meta path: /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl
Rows: 10710
['Book Id', 'Title', 'Author', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher', 'genres', 'year', 'retrieval_text']
Embedding shape: (10710, 384)
Saved FAISS index to /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Saved metadata to /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl
✅ Rebuild complete!
